In [1]:
import sys
sys.path.append('../')
import importlib
import models.transunet as vit
import experiments.config as conf
# import data_processing.dataset_synapse as dp 
import data_processing.data_parser as dp
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf 
import tensorflow_addons as tfa

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [27]:
importlib.reload(dp)
dr = dp.DataReader("/home/bouzidkenza1/TransUnet/data/synapse-tfrecords/")

In [28]:
dataset = dr.get_dataset()

In [ ]:
for sample in dataset.take(10):
    print(sample[0].shape)
    plt.imshow(sample[0])
    plt.show()

In [19]:
# %%time
# data_dir = "/home/bouzidkenza1/TransUnet/data/train_npz/"
# X, y = dp.load_data(data_dir, 2211)

In [18]:
config = conf.get_b16_none()
trans = vit.TransUnet(config)
trans.compile()

../models/utils.py:84: UserWarning: Resizing position embeddings from 24 to 32
  UserWarning,


In [21]:
data_batch = dataset.batch(30)

In [22]:
%%time
trans.model.fit(data_batch, epochs=3, batch_size=32, verbose=1)

Epoch 1/3


ValueError: Creating variables on a non-first call to a function decorated with tf.function.